# Convert the unformatted Glaive dataset to ShareGPT


In [3]:
%load_ext autoreload
%autoreload 2
import lilac as ll

ll.set_project_dir('./data')

if not ll.has_dataset('local', 'glaive-function-calling-v2'):
  ll.from_huggingface(
    'glaiveai/glaive-function-calling-v2',
    'local',
    'glaive-function-calling-v2',
  )

ds = ll.get_dataset('local', 'glaive-function-calling-v2')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import re

ROLES = ['USER', 'ASSISTANT', 'FUNCTION RESPONSE']

# The split regex is a role, plus semicolon and space. For example
# "USER: " or "FUNCTION RESPONSE: ".
split_re = re.compile(r'({}): '.format('|'.join(ROLES)))


def _parse_chat(row: dict):
  system_prompt = row.get('system')

  chat = row['chat']
  # Split chat by split_res, and remove empty strings.
  chats = [s.strip() for s in split_re.split(chat) if s]

  # results look like:
  # ['USER', 'Can you book a flight for me from New York to London?', 'ASSISTANT', '...']
  # We now want it to be a dictionary of {'from': 'user', 'value': 'Can you book a flight...'}
  chats = [{'from': role.lower(), 'value': value} for role, value in zip(chats[::2], chats[1::2])]

  if system_prompt:
    chats = [{'from': 'system', 'value': system_prompt}] + chats

  return chats


res = ds.map(_parse_chat, output_path='conversations', overwrite=True)

[local/glaive-function-calling-v2][1 shards] map "_parse_chat" to "('conversations',)":   0%|          | 0/112960 [00:00<?, ?it/s]

[local/glaive-function-calling-v2][1 shards] map "_parse_chat" to "('conversations',)": 100%|██████████| 112960/112960 [00:07<00:00, 15708.45it/s]


Wrote map output to conversations-00000-of-00001.parquet


In [6]:
hf_ds = ds.to_huggingface()

print(hf_ds)

hf_ds.push_to_hub('lilacai/glaive-function-calling-v2-sharegpt')

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['chat', 'system', '__hfsplit__', 'chat__cluster', 'conversations'],
    num_rows: 112960
})


Pushing dataset shards to the dataset hub: 100%|██████████| 2/2 [00:21<00:00, 10.92s/it]
